In [7]:
# !pip install aider
!pip install intervaltree

In [8]:
import os
import sys
from pathlib import Path

from aider.dump import dump
from aider.io import InputOutput
from aider import models
from aider.repomap import RepoMap, Tag, find_src_files

from tree_sitter_languages import get_language, get_parser

In [9]:
Path.cwd()

PosixPath('/Users/jim/Projects/aider')

In [10]:
fnames = ['tests/test_coder.py']
# fnames = ['aider/coders/base_coder.py']
chat_fnames = []
# other_fnames = ['aider/coders/base_coder.py']
# other_fnames = ['tests/test_coder.py']
other_fnames = find_src_files('aider')
for fname in fnames:
    if Path(fname).is_dir():
        chat_fnames += find_src_files(fname)
    else:
        chat_fnames.append(fname)
dump(chat_fnames)
dump(other_fnames)
rm = RepoMap(root=".", io=InputOutput(), main_model=models.Model.create('gpt-4-1106-preview'))
repo_map = rm.get_ranked_tags_map(chat_fnames, other_fnames)

print(type(repo_map))
dump(len(repo_map))
dump(repo_map)

chat_fnames:
[
    "tests/test_coder.py"
]
other_fnames:
[
    "aider/repomap.py",
    "aider/versioncheck.py",
    "aider/sendchat.py",
    "aider/io.py",
    "aider/__init__.py",
    "aider/diffs.py",
    "aider/prompts.py",
    "aider/dump.py",
    "aider/voice.py",
    "aider/lsp.py",
    "aider/utils.py",
    "aider/repo.py",
    "aider/main.py",
    "aider/commands.py",
    "aider/history.py",
    "aider/coders/single_wholefile_func_coder.py",
    "aider/coders/refactor_func_coder.py",
    "aider/coders/wholefile_func_prompts.py",
    "aider/coders/single_wholefile_func_prompts.py",
    "aider/coders/wholefile_func_coder.py",
    "aider/coders/__init__.py",
    "aider/coders/editblock_func_prompts.py",
    "aider/coders/wholefile_coder.py",
    "aider/coders/base_coder.py",
    "aider/coders/wholefile_prompts.py",
    "aider/coders/refactor_func_prompts.py",
    "aider/coders/editblock_func_coder.py",
    "aider/coders/base_prompts.py",
    "aider/coders/editblock_prompts.py",
  

In [11]:
dump(chat_fnames[:10])
dump(other_fnames[:10])
dump(rm.get_ranked_tags_map(chat_fnames=chat_fnames, other_fnames=other_fnames))

chat_fnames[:10]:
[
    "tests/test_coder.py"
]
other_fnames[:10]:
[
    "aider/repomap.py",
    "aider/versioncheck.py",
    "aider/sendchat.py",
    "aider/io.py",
    "aider/__init__.py",
    "aider/diffs.py",
    "aider/prompts.py",
    "aider/dump.py",
    "aider/voice.py",
    "aider/lsp.py"
]
rm.get_ranked_tags_map(chat_fnames=chat_fnames, other_fnames=other_fnames):

aider/coders/base_coder.py:
⋮...
│def wrap_fence(name):
⋮...
│class Coder:
│    abs_fnames = None
⋮...
│    @classmethod
│    def create(
│        self,
│        main_model=None,
│        edit_format=None,
│        io=None,
│        skip_model_availabily_check=False,
│        **kwargs,
⋮...
│    def abs_root_path(self, path):
⋮...
│    def run(self, with_message=None):
⋮...
│    def fmt_system_prompt(self, prompt):
⋮...
│    def check_for_file_mentions(self, content):
⋮...
│    def get_rel_fname(self, fname):
⋮...
│    def get_inchat_relative_files(self):
⋮...
│    def get_all_relative_files(self):
⋮...
│    def ge

In [12]:
lang = 'python'
language = get_language(lang)
parser = get_parser(lang)

io = InputOutput()

lines = []

def get_tree(fname):
    global lines
    code = io.read_text(fname)
    if not code:
        return None
    lines = code.split("\n")
    return (parser.parse(bytes(code, "utf-8")))

In [13]:
tree = get_tree('tests/test_coder.py')
dump(tree)

tree: <tree_sitter.Tree object at 0x10bd38ab0>


In [14]:
tree.included_ranges

[<Range start_point=(0, 0), start_byte=0, end_point=(4294967295, 4294967295), end_byte=4294967295>]

In [15]:
code_text = '''
            # Initialize the Coder object with the mocked IO and mocked repo
            coder = Coder.create(models.GPT4, None, mock_io)

            # Call the check_for_file_mentions method
            coder.check_for_file_mentions("Please check file1.txt and file2.py")

            # Check if coder.abs_fnames contains both files
            expected_files = set(
                [
                    str(Path(coder.root) / fname1),
                    str(Path(coder.root) / fname2),
                ]
            )
'''

code_lines = code_text.split("\n")
# format line number as 4 digits with leading zeros
# line_num_format = '{:04d}'
print('\n'.join([f'{line_num:04d}:{line}' for line_num, line in enumerate(code_lines, start=101)]))

0101:
0102:            # Initialize the Coder object with the mocked IO and mocked repo
0103:            coder = Coder.create(models.GPT4, None, mock_io)
0104:
0105:            # Call the check_for_file_mentions method
0106:            coder.check_for_file_mentions("Please check file1.txt and file2.py")
0107:
0108:            # Check if coder.abs_fnames contains both files
0109:            expected_files = set(
0110:                [
0111:                    str(Path(coder.root) / fname1),
0112:                    str(Path(coder.root) / fname2),
0113:                ]
0114:            )
0115:


In [16]:
dump(len(lines))

len(lines): 581


In [17]:
rm.tokenizer

<Encoding 'cl100k_base'>

In [21]:
def node_text(node):
    return node.text.decode("utf-8")

def node_display(node, lines):
    global rm
    # print(node.type)
    # if ('definition' in node.type) or ('reference' in node.type):
    if 'function_definition' == node.type:
        # t = node_text(node)
        # lines = t.split('\n')
        numbered_lines = '\n'.join([f'{line_num:04d}:{line}' for line_num, line in enumerate(lines[node.start_point[0]:node.end_point[0]+1], start=node.start_point[0]+1)])
        tokens = rm.tokenizer.encode(numbered_lines)
        print(f'{node.grammar_name} @ [{node.start_point[0]+1}..{node.end_point[0]+1}] len: {len(numbered_lines)} tokens: {len(tokens)}')
        # dump(tokens)
        token_bytes = [rm.tokenizer.decode_single_token_bytes(token) for token in tokens]
        print(token_bytes)
        # print(f'{node.grammar_name} @ {node.start_point[0]+1}..{node.end_point[0]+1}: {lines[0]}')
        # print(node.sexp())
        # print(t)
        # print('\n'.join(zip(enumerate(lines[node.start_point[0]:node.end_point[0]+1], start=node.start_point[0]+1))))
        # dump(code_lines[node.start_point[0]:node.end_point[0]+1])
        # print('\n'.join([f'{line_num:04d}:{line}' for line_num, line in enumerate(lines, start=node.start_point[0]+1)]))
        print(numbered_lines)
        print()
    for child in node.children:
        node_display(child, lines)

node_display(tree.root_node, lines)

function_definition @ [17..20] len: 212 tokens: 54
[b'001', b'7', b':', b'   ', b' def', b' setUp', b'(self', b'):\n', b'001', b'8', b':', b'       ', b' self', b'.patch', b'er', b' =', b' patch', b'("', b'a', b'ider', b'.cod', b'ers', b'.base', b'_c', b'oder', b'.check', b'_model', b'_av', b'ailability', b'")\n', b'001', b'9', b':', b'       ', b' self', b'.mock', b'_check', b' =', b' self', b'.patch', b'er', b'.start', b'()\n', b'002', b'0', b':', b'       ', b' self', b'.mock', b'_check', b'.return', b'_value', b' =', b' True']
0017:    def setUp(self):
0018:        self.patcher = patch("aider.coders.base_coder.check_model_availability")
0019:        self.mock_check = self.patcher.start()
0020:        self.mock_check.return_value = True

function_definition @ [22..23] len: 61 tokens: 17
[b'002', b'2', b':', b'   ', b' def', b' tearDown', b'(self', b'):\n', b'002', b'3', b':', b'       ', b' self', b'.patch', b'er', b'.stop', b'()']
0022:    def tearDown(self):
0023:        self.patc

In [19]:
query_scm_python = '''(class_definition
  name: (identifier) @name.definition.class) @definition.class

(function_definition
  name: (identifier) @name.definition.function) @definition.function

(call
  function: [
      (identifier) @name.reference.call
      (attribute
        attribute: (identifier) @name.reference.call)
  ]) @reference.call
'''

query = language.query(query_scm_python)
captures = query.captures(tree.root_node)

captures = list(captures)

In [20]:
for node, tag in captures:
    if ('definition' in tag) or ('reference' in tag):
        t = node_text(node)
        # Get the first line of the node
        t0 = t.split('\n')[0]
        print(f'{tag} @ {node.start_point[0]+1}..{node.end_point[0]+1}:')
        print(t0)
        # print(node.sexp())
        if 'reference' in tag:
            print(lines[(node.start_point[0]):node.end_point[0]+1])

definition.class @ 16..576:
class TestCoder(unittest.TestCase):
name.definition.class @ 16..16:
TestCoder
definition.function @ 17..20:
def setUp(self):
name.definition.function @ 17..17:
setUp
reference.call @ 18..18:
patch("aider.coders.base_coder.check_model_availability")
['        self.patcher = patch("aider.coders.base_coder.check_model_availability")']
name.reference.call @ 18..18:
patch
['        self.patcher = patch("aider.coders.base_coder.check_model_availability")']
reference.call @ 19..19:
self.patcher.start()
['        self.mock_check = self.patcher.start()']
name.reference.call @ 19..19:
start
['        self.mock_check = self.patcher.start()']
definition.function @ 22..23:
def tearDown(self):
name.definition.function @ 22..22:
tearDown
reference.call @ 23..23:
self.patcher.stop()
['        self.patcher.stop()']
name.reference.call @ 23..23:
stop
['        self.patcher.stop()']
definition.function @ 25..50:
def test_allowed_to_edit(self):
name.definition.function @ 25..25